# Using Natural Language Processing to derive insights from BSc Data Science personal statements

This contains 10 BSc Data Science personal statements representating 50% of the cohort starting in 2022

Using NPL the following things will be identified from the personal statements:
- Main topics (Using Topic Modelling)
- Top 10 most frequent words - this may be more insightful than topic modelling as topic modelling is an unsupervised machine learning algorithm it is not always as effective as desired 
- Key people who have been mentioned - this could be used to provide a basis for potential students to do further research to explore their interests
- Readability levels - This will be calculated using the Flesch-Kincaid Grade Level Forumla which calculates the grade level needed to understand a text using the average syllables per word and the average number of words per sentence. 

In [16]:
import pandas as pd

# Reading the text file
with open('ST115_personal_statements.txt', 'r') as file:
    all_ps = file.read()

# Displaying the first 1000 characters of the text file
all_ps[:1000]

'Personal statement:\nTaking part in a coding competition and several UKMT challenges exposed me to new ways of approaching problems from what I was used to in school. The challenge of having to think differently about problems and eventually arrive at solutions after reaching dead-ends was something I relished doing, which convinced me that a quantitative degree was for me.\nI have continued to challenge myself and explore the interplay between computer science and mathematics by solving Project Euler problems. For example, I used recursion to compute the convergents of continued fractions of the square root of 2. These problems nurtured my interest in pure mathematics, leading me to read Kevin Houston\'s "How to Think like a Mathematician". This introduced me to more sophisticated inductive proof techniques than the ones I learned at A-level. To supplement this, I explored how Lean, a programming language, can be taught to inductively prove a simple addition from first principles. Wh

Looking at the first 1000 characters shown above it can be seen that some intial data cleaning must be done in the sense of removing '\n' and any backslashes in general.
Also we want to manipulate this text data in order to store all the different personal statements as one text to do topic modelling with and to store them as individual statements to calculate readability and key people.

In [17]:
#Inital removal of '\n' and backslashes
all_ps = all_ps.replace("\n", "")
all_ps = all_ps.replace("\xa0", "")
all_ps = all_ps.replace("\'", "")

In [18]:
full_text = all_ps.replace("Personal statement:", " ")

In [19]:
all_ps

'Personal statement:Taking part in a coding competition and several UKMT challenges exposed me to new ways of approaching problems from what I was used to in school. The challenge of having to think differently about problems and eventually arrive at solutions after reaching dead-ends was something I relished doing, which convinced me that a quantitative degree was for me.I have continued to challenge myself and explore the interplay between computer science and mathematics by solving Project Euler problems. For example, I used recursion to compute the convergents of continued fractions of the square root of 2. These problems nurtured my interest in pure mathematics, leading me to read Kevin Houstons "How to Think like a Mathematician". This introduced me to more sophisticated inductive proof techniques than the ones I learned at A-level. To supplement this, I explored how Lean, a programming language, can be taught to inductively prove a simple addition from first principles. What ent

<b> STEP 1: Readability </b>
- Creating a dataframe where each row accounts for a single personal statement. This will be used to calculate the individual readability scores.

In [36]:
# This phrase has been used in the text file to indicate the start of a different personal statement
individual_statements = all_ps.split('Personal statement:')

ps_df = pd.DataFrame({'Personal Statements': individual_statements})

#The first row is empty so it is removed
ps_df = ps_df.drop(index=0).reset_index(drop=True)


Creating the Flesch-Kincaid function to calculate the readability scores 

In [35]:
import re

def count_syllables(word):
    '''
    Helper function to count the number of sylables in a given word
    Input: 
    word : string
    
    Output:
    num_vowels : int
    '''
    num_vowels = len(re.findall(r'[aeiouy]+', word))

    # Subtract the number of silent e's at the end of the word
    if re.search(r'e$', word):
        num_vowels -= 1

    # Subtract one for each diphthong
    num_vowels -= len(re.findall(r'[aeiouy]{2}', word))

    # Add one if the word ends in "-le"
    if re.search(r'le$', word):
        num_vowels += 1

    # Add one if the word is one syllable and ends in a consonant followed by "y"
    if len(re.findall(r'^[^aeiouy]+[aeiouy]+[^aeiouy]+y$', word)):
        num_vowels += 1

    return max(1, num_vowels)

def flesch_kincaid_grade(text):
    '''
    The function uses the Flesch-Kincaid formula to calculate the grade level required to be able to read a specificed text
    
    Inputs: 
    text : string
    The text to calculate the readability level of 
    
    Outputs: 
    grade_level : float
    Grade age of readability 
    '''
    text = text
    sentences = text.split('.')
    words = text.split()

    # Calculate the average number of words per sentence
    words_per_sentence = len(words) / len(sentences)

    # Calculate the average number of syllables per word
    syllables = 0
    for word in words:
        syllables += count_syllables(word)
    syllables_per_word = syllables / len(words)

    grade_level = 0.39 * words_per_sentence + 11.8 * syllables_per_word - 15.59
    
    grade_level = round(grade_level, 1)

    return grade_level

Applying the function to all the personal statements

In [37]:
ps_df['Readability Age Grade'] = ps_df['Personal Statements'].apply(flesch_kincaid_grade)


In [38]:
ps_df

,Personal Statements,Readability Age Grade
0,Taking part in a coding competition and severa...,12.5
1,Mathematics and the social sciences are seen a...,14.5
2,Data Science appealed to me as it offers a ne...,14.0
3,My interest in data analytics stems from an ex...,13.4
4,"Growing up on the football pitch, I learnt th...",10.6
5,"I have always been drawn to figures and data, ...",12.1
6,Data is one of the most valuable tools that th...,11.8
7,I couldn’t believe what I saw. After looking a...,14.0
8,I believe that the intersection of mathematic...,13.7
9,Seeing how influential big data is in shaping...,13.7


<b> STEP 2: NLP Preprocessing tasks</b> 

The following tasks will be carried out in this section: 
- Tokenisation
- Stop word removal
- Lemmatisation
- POS tagging
- Named Entity Recognition 
- TF-IDF
    

<b>Tokenisation:</b>

In [78]:
import nltk
from nltk.tokenize import word_tokenize

full_text = full_text.lower()

#Creating a dataframe to store all the steps of preprocessing
nlp_df = pd.DataFrame({'Original text': full_text}, index = [0])

tokens = word_tokenize(full_text)
nlp_df['Tokens'] = [tokens]


<b> Removing stop words </b>

In [79]:
from nltk.corpus import stopwords
sw = stopwords.words('English')


In [80]:
filtered_tokens = [token for token in tokens if token not in sw]


Checking if there are any words that are still in tokens and not in the NLTK stopwords that should be taken out

In [81]:
full_token_dict = {}

for token in filtered_tokens:
    if token in full_token_dict: 
        full_token_dict[token] += 1
    else: 
        full_token_dict[token] = 1

sorted_dict = sorted(full_token_dict.items(), key=lambda x: x[1], reverse=True)

sorted_dict[:10]


[(',', 246),
 ('.', 219),
 ('data', 106),
 ('’', 42),
 ('learning', 41),
 ('science', 36),
 ('using', 24),
 ('machine', 23),
 ('used', 21),
 ('social', 17)]

As can be seen the only tokens that were not in stop words that should be taken out are punctuation

In [99]:
#Taking out punctuation
filtered_tokens = [token for token in filtered_tokens if token not in ['.', ',', "'", '’', '‘', '[', ']', ':', "''", '``']]

nlp_df['Filtered Tokens'] = [filtered_tokens]

In [100]:
filtered_tokens

['taking',
 'part',
 'coding',
 'competition',
 'several',
 'ukmt',
 'challenges',
 'exposed',
 'new',
 'ways',
 'approaching',
 'problems',
 'used',
 'school',
 'challenge',
 'think',
 'differently',
 'problems',
 'eventually',
 'arrive',
 'solutions',
 'reaching',
 'dead-ends',
 'something',
 'relished',
 'convinced',
 'quantitative',
 'degree',
 'me.i',
 'continued',
 'challenge',
 'explore',
 'interplay',
 'computer',
 'science',
 'mathematics',
 'solving',
 'project',
 'euler',
 'problems',
 'example',
 'used',
 'recursion',
 'compute',
 'convergents',
 'continued',
 'fractions',
 'square',
 'root',
 '2.',
 'problems',
 'nurtured',
 'interest',
 'pure',
 'mathematics',
 'leading',
 'read',
 'kevin',
 'houstons',
 'think',
 'like',
 'mathematician',
 'introduced',
 'sophisticated',
 'inductive',
 'proof',
 'techniques',
 'ones',
 'learned',
 'a-level',
 'supplement',
 'explored',
 'lean',
 'programming',
 'language',
 'taught',
 'inductively',
 'prove',
 'simple',
 'addition',
 'fi

<b> Lemmatisation </b>

In [88]:
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ekmho\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ekmho\AppData\Roaming\nltk_data...


True

In [101]:
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

lemmatized_tokens = []

for token in filtered_tokens:
    lemmatized_token = lemmatizer.lemmatize(token)
    lemmatized_tokens.append(lemmatized_token)

In [102]:
lemmatized_tokens

['taking',
 'part',
 'coding',
 'competition',
 'several',
 'ukmt',
 'challenge',
 'exposed',
 'new',
 'way',
 'approaching',
 'problem',
 'used',
 'school',
 'challenge',
 'think',
 'differently',
 'problem',
 'eventually',
 'arrive',
 'solution',
 'reaching',
 'dead-ends',
 'something',
 'relished',
 'convinced',
 'quantitative',
 'degree',
 'me.i',
 'continued',
 'challenge',
 'explore',
 'interplay',
 'computer',
 'science',
 'mathematics',
 'solving',
 'project',
 'euler',
 'problem',
 'example',
 'used',
 'recursion',
 'compute',
 'convergents',
 'continued',
 'fraction',
 'square',
 'root',
 '2.',
 'problem',
 'nurtured',
 'interest',
 'pure',
 'mathematics',
 'leading',
 'read',
 'kevin',
 'houston',
 'think',
 'like',
 'mathematician',
 'introduced',
 'sophisticated',
 'inductive',
 'proof',
 'technique',
 'one',
 'learned',
 'a-level',
 'supplement',
 'explored',
 'lean',
 'programming',
 'language',
 'taught',
 'inductively',
 'prove',
 'simple',
 'addition',
 'first',
 'pri